In [1]:
import Pyro5.api
import os
from glob import glob
import json

from imagedaemon.utils.paths import RAW_DATA_DIR


ns = Pyro5.api.locate_ns(host="localhost")

#winter = Pyro5.api.Proxy(ns.lookup("winter_daemon"))
#wcs = winter.get_astrometry(addr="pb", science_image="IMG.fits", ra=..., dec=...)

qcmos = Pyro5.api.Proxy(ns.lookup("qcmos_daemon"))

winter = Pyro5.api.Proxy(ns.lookup("winter_daemon"))

_PROJECT_ROOT = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/winter-image-daemon/imagedaemon
CONFIG_DIR = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/winter-image-daemon/imagedaemon/imagedaemon/config
astro_scamp = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/winter-image-daemon/imagedaemon/imagedaemon/config/scamp.conf
masterbias_dir = /Users/nlourie/data/image-daemon-data/calibration/masterbias


In [2]:
# ask the daemon to analyze a qcmos image

image_dir = os.path.join(RAW_DATA_DIR, "qcmos", "science")
science_images = sorted(glob(os.path.join(image_dir, "*8s*.fits")))
science_image = science_images[0]
print(f"Science image: {science_image}")

# run the full astrometric pipeline
output_dir = os.path.join(os.path.join(os.getenv("HOME"), "data", "tmp"))

# this image has no RA/Dec in the header, so we need to set it
ra_guess = 83.6
dec_guess = 22



astrom_info = qcmos.solve_astrometry(science_image = science_image,
                                      background_image_list = None,
                                      output_dir = output_dir,
                                      ra = ra_guess,
                                      dec = dec_guess,
                                      pix_coords = None,
    )

print(f"Astrometric solution: {json.dumps(astrom_info, indent = 2)}")



Science image: /Users/nlourie/data/image-daemon-data/raw/qcmos/science/8s_image_single.fits
Astrometric solution: {
  "ra": 83.63355276039101,
  "dec": 21.998954213523806,
  "pix_coords": [
    2048,
    1152
  ],
  "pixel_scale": 0.15731235843477753,
  "rotation_deg": 179.99645031338304,
  "image_width": 4096,
  "image_height": 2304
}


In [ ]:
# ask the daemon to analyze a winter image

addr = "pb"
bkg_dir = os.path.join(RAW_DATA_DIR, "winter", "science", "30s", "bkg")
science_dir = os.path.join(RAW_DATA_DIR, "winter", "science", "30s")

bkg_images = sorted(glob(os.path.join(bkg_dir, "*mef.fits")))
MAX_BKG_IMAGES = 3

bkg_images = bkg_images[:MAX_BKG_IMAGES]
science_images = sorted(glob(os.path.join(science_dir, "*mef.fits")))
science_image = science_images[0]

use_science_in_bkg = True

if use_science_in_bkg:
    bkg_images = [science_image] + bkg_images
    print(f"Using {len(bkg_images)} background images with science image")
else:
    print(f"Using {len(bkg_images)} background images without science image")

output_dir = os.path.join(os.path.join(os.getenv("HOME"), "data", "tmp"))
output_dir = "tmp" # so that it doesn't save any extra files

astrom_info = winter.solve_astrometry(addr=addr,
                                    science_image=science_image,
                                    background_image_list=bkg_images,
                                    output_dir=output_dir,
                                    pix_coords=None,
                                    ra = 10,
                                    dec = 10,
                                    timeout=10,
)

print(f"Astrometric solution: {json.dumps(astrom_info, indent = 2)}")




Using 4 background images with science image


RuntimeError: solve‑field exceeded 30s